# Contoso Sales Analysis Assistant


## Why use the OpenAI Assistants API

The OpenAI Assistants API allows you to build conversational agents that can understand and respond to user inputs. You can use the API to automate tasks, provide information, or guide users through a process.

The Assistants API is not the only way to build conversational agents, but it offers several advantages:

1. Simplicity: The API abstracts away the complexity of building a conversational agent, allowing you to focus on the content and logic of the conversation.
2. Scalability: The API is designed to handle a large number of concurrent users, making it suitable for production use.
3. Customization: The API allows you to customize the behavior of the assistant by providing training data and defining conversational flows.
4. Integration: The API can be integrated with other services and systems, allowing you to build assistants that interact with external data sources.


## Core concepts

The OpenAI Assistants API is based on a few core concepts that you need to understand in order to build conversational agents:

### Conversational agents

A conversational agent is a computer program that can interact with users in a natural language. The agent can understand user inputs, generate responses, and maintain a context of the conversation.

### Conversational flows

A conversational flow is a sequence of interactions between the user and the assistant. The flow can include questions, prompts, and responses that guide the user through a process or provide information.

### The Assistant object

The Assistant object is the main interface for interacting with the Assistants API. You can create an Assistant object by providing your API key and the ID of the assistant you want to use.

### The Thread object

The Thread object represents a conversation thread between the user and the assistant.

### Function calling

The Assistants API allows you to define custom functions that can be called from within the conversation. These functions can perform tasks, retrieve data, or interact with external services.

### Code Generation

The Assistants API can generate Python code to be executed in a sandboxed environment. This code can be used to define custom functions, manipulate data, or perform calculations. The generated code is executed in a secure environment to prevent malicious behavior.


## Objective

This notebook demonstrates the following:

1. Generative AI
1. Function calling
1. Code Generation

Reference:

- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant)
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)

<!-- - Function Calling with Yfinance to get latest stock prices. Summarization of user provided article. Extract country info from article, extract country, capital and other aspects, and call an API to get more information about each country.

This tutorial uses the following Azure AI services:
- Access to Azure OpenAI Service - you can apply for access [here](https://aka.ms/oai/access)
- Azure OpenAI service - you can create it from instructions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource)
- Azure OpenAI Studio - go to [https://oai.azure.com/](https://oai.azure.com/) to work with the Assistants API Playground
- A connection to the Azure OpenAI Service with a [Key and Endpoint](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart)

-->


## Time

You should expect to spend 10-15 minutes running this sample.


## About this example

The objective of the provided Python file is to create an Azure OpenAI Assistant named "Contoso Sales Analysis Assistant" using the Azure OpenAI API. The assistant is designed to act as a sales analysis assistant, providing information and insights related to the contoso business sales. The script initiates a conversation with the assistant, guiding it through various financial queries and scenarios to showcase its capabilities.

### Data

This sample uses files from the folder [`data/`](./data/) in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.


## Installation

Refer to the README.md file in this folder for installation instructions.


In [1]:
# Install the packages
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Parameters


In [2]:
import os
import json
import sqlite3
import pandas as pd
from IPython.display import display
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(".env")
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

con = sqlite3.connect("./database/order_data.db")

In [3]:
def get_revenue_by_region(region: str) -> pd.DataFrame:
    if region:
        query = f"""
            SELECT 
                Region,
                SUM(Number_of_Orders) as Number_of_Orders, 
                CAST(SUM(Sum_of_Order_Value_USD) AS INTEGER) as Total_Order_Value,
                SUM(Sum_of_Order_Value_USD)/SUM(Number_of_Orders) as Avg_Order_Value,
                SUM(Sum_of_Shipping_Cost_USD)/SUM(Number_of_Orders) as Avg_Shipping_Cost
            FROM order_data 
            WHERE UPPER(Region) = '{region.upper()}'
            GROUP BY Region
        """
    else:
        query = """
            SELECT 
                Region,
                SUM(Number_of_Orders) as Number_of_Orders, 
                CAST(SUM(Sum_of_Order_Value_USD) AS INTEGER) as Total_Order_Value,
                SUM(Sum_of_Order_Value_USD)/SUM(Number_of_Orders) as Avg_Order_Value,
                SUM(Sum_of_Shipping_Cost_USD)/SUM(Number_of_Orders) as Avg_Shipping_Cost
            FROM order_data 
            GROUP BY Region ORDER BY Region
        """
    return pd.read_sql_query(query, con)

### Create an AzureOpenAI client


In [4]:
client = AzureOpenAI(
    api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint
)

### Define the Assistant tools


In [5]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "get_revenue_by_region",
            "description": "Get the sales revenue for Contoso by region.",
            "parameters": {
                "type": "object",
                "properties": {
                    "region": {"type": "string"},  # Example parameter
                },
            },
        },
    }
]

### Process Function calling


In [6]:
def call_functions(tool_calls) -> None:

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        region = function_args.get("region")

        print(f"\nFunction Name: {function_name}, Function Args: {function_args}\n\n")

        df = get_revenue_by_region(region=region)
        print(df.to_string(index=False))

In [7]:
def process_message(question: str):

    instructions = " ".join(
        (
            "You are a sales analysis assistant for Contoso.",
            "Please be polite, professional, helpful, and friendly.",
            "You get all the sales data from this app using the functions provided.",
            "If a question is not related to the sales or you cannot answer the question, ",
            "say, 'contact IT for more assistance.' ",
            "If the user asks for help or says 'help', provide a list of sample questions that you can answer.",
        )
    )

    messages = [
        {
            "role": "system",
            "content": instructions,
        },
        {"role": "user", "content": question},
    ]

    response = client.chat.completions.create(
        model=api_deployment_name,
        messages=messages,
        tools=tools_list,
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        call_functions(tool_calls)
    else:
        display(f"Response message: {response_message.content}")
        return

In [8]:
process_message("What is the revenue for the region of Africa?")


Function Name: get_revenue_by_region, Function Args: {'region': 'Africa'}


Region  Number_of_Orders  Total_Order_Value  Avg_Order_Value  Avg_Shipping_Cost
AFRICA             61209            1533671        25.056306           7.507702


In [9]:
process_message("What is the revenue for the region of north america?")


Function Name: get_revenue_by_region, Function Args: {'region': 'north america'}


       Region  Number_of_Orders  Total_Order_Value  Avg_Order_Value  Avg_Shipping_Cost
NORTH AMERICA             62555            1092029        17.457103           7.502805


In [10]:
process_message("What is the revenue for all regions?")


Function Name: get_revenue_by_region, Function Args: {}


       Region  Number_of_Orders  Total_Order_Value  Avg_Order_Value  Avg_Shipping_Cost
       AFRICA             61209            1533671        25.056306           7.507702
 ASIA-PACIFIC             61337            1344150        21.914194           7.490754
       EUROPE             61719            1199818        19.440016           7.502063
  MIDDLE EAST             61168            1786876        29.212609           7.502444
NORTH AMERICA             62555            1092029        17.457103           7.502805
SOUTH AMERICA             61193            2147262        35.090006           7.475050


In [11]:
process_message("What is the meaning of life?")

"Response message: That's a philosophical question that has been debated for centuries and can have many different interpretations depending on one's beliefs, values, and perspectives. For sales-related inquiries, feel free to ask about our revenue or other business metrics. For other questions, please contact IT for more assistance."